### Ejercicio 1.
Extraer los siguentes estadísiticos del conjunto de datos para varias de las varibles disponibles:
* Tabla de contingencia
* Coeficiente de correlación de las variables numéricas del dataset.
* Sesgos de alguna de las variables numéricas del dataset.
* Intervalo de confianza de algunas de las variables numéricas del dataset

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import scipy.stats as st
import re

In [2]:
df_influencers = pd.read_csv('../Archivos/instagram_global_top_1000.csv')
df_influencers.head(3)

,Country,Rank,Account,Title,Link,Category,Followers,Audience Country,Authentic engagement,Engagement avg,Scraped
0,All,1,cristiano,Cristiano Ronaldo,https://www.instagram.com/cristiano/,Sports with a ball,400100000.0,India,7800000.0,9500000.0,2022-02-07 16:50:24.798803
1,All,2,kyliejenner,Kylie 🤍,https://www.instagram.com/kyliejenner/,Fashion|Modeling|Beauty,308800000.0,United States,6200000.0,10100000.0,2022-02-07 16:50:24.798803
2,All,3,leomessi,Leo Messi,https://www.instagram.com/leomessi/,Sports with a ball|Family,306300000.0,Argentina,4800000.0,6500000.0,2022-02-07 16:50:24.798803


In [3]:
#exploramos las columnas y el tipo de datos que contienen
df_influencers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Country               1000 non-null   object 
 1   Rank                  1000 non-null   int64  
 2   Account               1000 non-null   object 
 3   Title                 983 non-null    object 
 4   Link                  1000 non-null   object 
 5   Category              909 non-null    object 
 6   Followers             1000 non-null   float64
 7   Audience Country      993 non-null    object 
 8   Authentic engagement  1000 non-null   float64
 9   Engagement avg        1000 non-null   float64
 10  Scraped               1000 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 86.1+ KB


In [6]:
df_influencers['Category'].isnull().sum()

91

In [7]:
#principales estadísticos de las variables numéricas:
df_influencers.describe()

,Rank,Followers,Authentic engagement,Engagement avg
count,1000.000000,1.000000e+03,1.000000e+03,1.000000e+03
mean,500.500000,2.483610e+07,6.491228e+05,8.653886e+05
std,288.819436,3.872143e+07,1.098804e+06,1.267515e+06
min,1.000000,2.800000e+06,0.000000e+00,5.200000e+04
25%,250.750000,9.000000e+06,2.091000e+05,3.086000e+05
50%,500.500000,1.440000e+07,3.637000e+05,5.239000e+05
75%,750.250000,2.565000e+07,6.832500e+05,9.508000e+05
max,1000.000000,4.696000e+08,1.340000e+07,1.340000e+07


     Vamos a crear una nueva columna llamada 'Continentes' para poder facilitar la legibilidad de la tabla de contingencias entre Países de audiencia x Tipo de contenido. Para ello:

In [10]:
#creación de la función
def unir_paises (col):
    EEUU = ['United States']
    Rusia = ['Russia']
    Latam = ['Argentina', 'Brazil', 'Mexico', 'Colombia', 'Chile']
    UE = ['France', 'Spain', 'Italy', 'Poland', 'Germany']
    Asia_SE = ['Indonesia', 'Japan', 'South Korea', 'Thailand', 'Philippines']
    Africa = ['Nigeria', 'Morocco', 'Algeria']
    Asia_arabe = ['Iran', 'Egypt', 'Turkey', 'Saudi Arabia', 'United Arab Emirates', 'Iraq', 'Syria', 'Kazakhstan']

    if col in Asia_SE:
        return 'Sudeste asiático'
    elif col in UE:
        return 'UE'
    elif col in Latam:
        return 'Latam'
    elif col in Africa:
        return 'Africa'
    elif col in Asia_arabe:
        return 'Países arabes'
    elif col in EEUU:
        return 'EEUU'
    elif col in Rusia:
        return 'Russia'

In [42]:
#aplicación de la función a la nueva columna
df_influencers['Continentes'] = df_influencers['Audience Country'].apply(unir_paises)
df_influencers['Continentes'].unique()

array([None, 'EEUU', 'Latam', 'Sudeste asiático', 'Países arabes', 'UE',
       'Russia', 'Africa'], dtype=object)

        La cantidad de categorías es abrumadora, por lo que vamos a crear otra función que nos permita condensar los tipos de contenidos creados en otras categorías más generales:

In [38]:
#creamos la función:
def clasif_contenido(col):
    Moda = ['Fashion', 'Accessories', 'Beauty', 'Modeling','Clothing','Outfits']
    Est_de_vida = ['Lifestyle']
    Arte = ['Arts', 'Cinema', 'Photography', 'Shows', 'Actors/actresses','Artists']
    Tecnologias = ['Computers','Gadgets','Machinery','Technologies']
    Inversion = ['Finance', 'Economics','Business','Careers']
    Deportes = ['Sports with a ball', 'Racing sports']
    try:
        if col in Deportes:
            return 'Deportes'
        if col in Moda:
            return 'Moda'
        if col in Est_de_vida:
            return 'Estilo de vida'
        if col in Arte:
            return 'Arte'
        if col in Tecnologias:
            return 'Tecnologías'
        if col in Inversion:
            return 'Inversión'
        else:
            return 'Otros'
    except:
        return np.nan

In [43]:
#aplicamos la función a la nueva clasificación de contenidos:
df_influencers['Cat_clasif'] = df_influencers['Category'].apply(clasif_contenido)
df_influencers['Cat_clasif'].unique()

array(['Deportes', 'Otros', 'Arte', 'Estilo de vida', 'Moda'],
      dtype=object)

In [41]:
#tabla de contingencias de las variables categóricas:
df_influencers_crosstab = pd.DataFrame(pd.crosstab(df_influencers['Continentes'], df_influencers['Cat_clasif']))
df_influencers_crosstab

Cat_clasif,Arte,Deportes,Estilo de vida,Moda,Otros
Continentes,,,,,
Africa,0,3,0,0,5
EEUU,5,11,13,9,245
Latam,4,22,18,12,189
Países arabes,4,6,2,3,34
Russia,6,0,2,2,20
Sudeste asiático,2,2,16,7,129
UE,0,36,2,2,21


In [7]:
#matriz de correlación
correlacion_influs = df_influencers.corr()
correlacion_influs

C:\Users\PcCom\AppData\Local\Temp\ipykernel_10204\3838001811.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlacion_influs = df_influencers.corr()


,Rank,Followers,Authentic engagement,Engagement avg
Rank,1.000000,-0.435286,-0.463701,-0.506682
Followers,-0.435286,1.000000,0.287067,0.336169
Authentic engagement,-0.463701,0.287067,1.000000,0.981982
Engagement avg,-0.506682,0.336169,0.981982,1.000000


* Intervalo de confianza:

    Vamos a calcular el intervalo de confianza de la variable 'Engagement auténtico'.

In [45]:
datos = df_influencers['Authentic engagement'] 

In [55]:
#Use of keyword argument `alpha` for method `interval` is deprecated. Use first positional argument or keyword argument `confidence` instead.
IC = st.t.interval(confidence=0.99, df=len(df_influencers)-1, loc=np.mean(datos), scale=st.sem(datos))
print(f'El valor que no conocemos de Authentic Engagement está entre {round(IC[0], 2)} e {round(IC[1], 2)} con una confianza del 99%')

El valor que no conocemos de Authentic Engagement está entre 559448.58 e 738797.02 con una confianza del 99%


### Ejercicio 2.

Interpretación de los resultados.

* Ahora interpreta los resultados obtenidos de los diferentes estadísticos estudiados haciendo hincapié en aspectos interesantes de tus datos y explica lo mejor posible la naturaleza de los mismos.

*Matriz de correlación:*

Observamos que existe una correlación positiva muy fuerte (0.98) entre las variables 'Authentic Engagement' y 'Engagement avg', lo cual nos indica que, a mayor interacción con el contenido, mayor cantidad de engagement real.

Existe también una correlación inversa moderada-alta (-0.50) entre la posición en el ranking y la media del engagement, por lo que a mayor posición en el ranking, menor será el engagement promedio.

Finalmente, resulta llamativa los resultados de la correlación entre la posición en el ranking ('Rank') y el número de seguidores sea, por un lado, moderada, y por otro, negativa (-0.44). A priori, esperaríamos que a mayor posición en el ranking de influencers con más seguidores, mayor número de seguidores. Es decir, que fuera una correlación positiva y fuerte. Sin embargo, no es el caso. Futuras aproximaciones a los datos deberán explorar qué otras variables pueden influir en esta relación, o si se ha producido un fallo en la recolección y/o volcado de los datos.

*Tabla de contingencias de las variables categóricas Audience Country x Category*:  

La mayoría de personas que siguen contenido de influencers deportivos son de países de la UE, seguidos de los países de latinoamérica y EEUU. En el caso de África y los países árabes, lxs influencers de deportes son lxs segundxs más seguidxs.  Por otro lado, lxs seguidorxs de países del sudeste asiático tienden a preferir contenidos relacionados con el 'Estilo de vida' y 'Moda'.  Lxs rusxs, por otro lado, parecen inclinarse mayoritariamente por contenidos relacionados con la estética y lo cultural como el arte y la moda.

La categoría 'Otros' es la predominante, lo que nos indica que en futuras aproximaciones deberemos realizar una disección más profunda de las categorías.